**IMPORT WYMAGANYCH BIBLIOTEK I BAZ DANYCH**

In [1]:
import numpy as np
import pandas as pd
import string
import nltk
import itertools
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marci\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marci\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marci\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\marci\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


**IMPORT DANYCH - MAIL (spam, nie spam)**

In [2]:
spam_dataset = pd.read_csv('spam.csv', encoding = "ISO-8859-1", usecols=[0, 1], names=['Spam', 'Text'],
                           skiprows=1)
spam_dataset['Spam'] = spam_dataset['Spam'].replace(['ham', 'spam'], [0, 1])

C:\Users\marci\AppData\Local\Temp\ipykernel_848\1552169415.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spam_dataset['Spam'] = spam_dataset['Spam'].replace(['ham', 'spam'], [0, 1])


**SERIA FUNKCJI i MODYFIKACJI MAIL:**

In [3]:
def remove_puncation(text):
    cleaned = ''.join([word for word in text if word not in string.punctuation])
    return cleaned

spam_dataset['Cleaned_Text'] = spam_dataset['Text'].apply(lambda x: remove_puncation(x))

In [4]:
def tokenize(text):
    clean_text = text.lower()
    tokenized_text = nltk.word_tokenize(clean_text)
    return tokenized_text

spam_dataset['Tokenized_Text'] = spam_dataset['Cleaned_Text'].apply(lambda x: tokenize(x))

In [5]:
stopwords = nltk.corpus.stopwords.words("english")

def remove_stopwords(text):
    without_stopwords = [word for word in text if word not in stopwords]
    return without_stopwords
spam_dataset['WithoutStop_Text'] = spam_dataset['Tokenized_Text'].apply(lambda x: remove_stopwords(x))

In [6]:
lemmater = nltk.WordNetLemmatizer()
def lemmatizing(text):
    lemmatized_words = [lemmater.lemmatize(word) for word in text]
    return lemmatized_words
spam_dataset['Lemmatized_Text'] = spam_dataset['WithoutStop_Text'].apply(lambda x: lemmatizing(x))
spam_dataset

,Spam,Text,Cleaned_Text,Tokenized_Text,WithoutStop_Text,Lemmatized_Text
0,0,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazy, available, bugis, n..."
1,0,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]"
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,0,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]","[u, dun, say, early, hor, u, c, already, say]"
4,0,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t...","[nah, dont, think, go, usf, life, around, though]"
...,...,...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,...","[2nd, time, tried, 2, contact, u, u, å£750, po...","[2nd, time, tried, 2, contact, u, u, å£750, po..."
5568,0,Will Ì_ b going to esplanade fr home?,Will Ì b going to esplanade fr home,"[will, ì, b, going, to, esplanade, fr, home]","[ì, b, going, esplanade, fr, home]","[ì, b, going, esplanade, fr, home]"
5569,0,"Pity, * was in mood for that. So...any other s...",Pity was in mood for that Soany other suggest...,"[pity, was, in, mood, for, that, soany, other,...","[pity, mood, soany, suggestions]","[pity, mood, soany, suggestion]"
5570,0,The guy did some bitching but I acted like i'd...,The guy did some bitching but I acted like id ...,"[the, guy, did, some, bitching, but, i, acted,...","[guy, bitching, acted, like, id, interested, b...","[guy, bitching, acted, like, id, interested, b..."


**STWORZENIE BAZY DANYCH KTÓRA BĘDZIE CIĄGIEM SŁÓW - FORMAT WYMAGANY PRZEZ CountVectorizer i TfidfVectorizer**

In [7]:
spam_dataset['Chain_Text'] = spam_dataset['Lemmatized_Text'].apply(lambda x: ' '.join(x))

X = spam_dataset['Chain_Text']
y = spam_dataset['Spam']

**Dzielimy zbiór X i y na treningowy i testowy ze stratyfikacją według y w proporcji 80/20**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=7)

**Tworzymy pipeline - model_pipeline_tfidf**

In [9]:
model_pipeline_tfidf = Pipeline(steps=[
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range=(2, 2), max_df=0.5, min_df=0.001, use_idf=True)),
    ('classifier_rf_tfidf', RandomForestClassifier(max_depth=2, random_state=7))
])

model_pipeline_tfidf.fit(X_train,y_train)

Pipeline(steps=[('vectorizer_tfidf',
                 TfidfVectorizer(max_df=0.5, min_df=0.001, ngram_range=(2, 2))),
                ('classifier_rf_tfidf',
                 RandomForestClassifier(max_depth=2, random_state=7))])

**Tworzymy pipeline - model_pipeline_count**

In [10]:
model_pipeline_count = Pipeline(steps=[
    ('vectorizer_count', CountVectorizer(ngram_range=(2, 2), max_df=0.5, min_df=0.001)),
    ('classifier_rf_count', RandomForestClassifier(max_depth=2, random_state=7))
])

model_pipeline_count.fit(X_train,y_train)

Pipeline(steps=[('vectorizer_count',
                 CountVectorizer(max_df=0.5, min_df=0.001, ngram_range=(2, 2))),
                ('classifier_rf_count',
                 RandomForestClassifier(max_depth=2, random_state=7))])

**Wyciągamy feature importance i dokonujemyselekcji cech - bierzemy tylko te których ważność jest większa niż 0.001**

In [11]:
classifier_tfidf = model_pipeline_tfidf.named_steps['classifier_rf_tfidf']
classifier_count = model_pipeline_count.named_steps['classifier_rf_count']

importances_tfidf = classifier_tfidf.feature_importances_
importances_count = classifier_count.feature_importances_

selected_features_tfidf = np.where(importances_tfidf > 0.001)[0]
selected_features_count = np.where(importances_count > 0.001)[0]

**Sprawdźmy jakie to cechy**

In [12]:
vectorizer_tfidf = model_pipeline_tfidf.named_steps['vectorizer_tfidf']
vectorizer_count = model_pipeline_count.named_steps['vectorizer_count']

feature_names_tfidf = vectorizer_tfidf.get_feature_names_out()
feature_names_count = vectorizer_count.get_feature_names_out()

df_importances_tfidf = pd.DataFrame({
    'feature': feature_names_tfidf,
    'importance': importances_tfidf
})

df_importances_count = pd.DataFrame({
    'feature': feature_names_count,
    'importance': importances_count
})

df_selected_tfidf = df_importances_tfidf[df_importances_tfidf['importance'] > 0.001].sort_values(by='importance', ascending=False)
df_selected_count = df_importances_count[df_importances_count['importance'] > 0.001].sort_values(by='importance', ascending=False)

print('Cechy o wadze więcej niż 0.001 dla - classifier_rf_tfidf')
print(df_selected_tfidf)
print('Cechy o wadze więcej niż 0.001 dla - classifier_rf_count')
print(df_selected_count)

Cechy o wadze więcej niż 0.001 dla - classifier_rf_tfidf
             feature  importance
287      please call    0.074337
163  guaranteed call    0.069498
4          1000 cash    0.041365
98    dating service    0.039669
71        chance win    0.038911
..               ...         ...
376    ufind rreveal    0.003350
380       ur awarded    0.003348
86   congrats mobile    0.003213
390   ur specialcall    0.003139
2           100 free    0.001803

[82 rows x 2 columns]
Cechy o wadze więcej niż 0.001 dla - classifier_rf_count
             feature  importance
287      please call    0.074337
163  guaranteed call    0.069498
4          1000 cash    0.041365
98    dating service    0.039669
71        chance win    0.038911
..               ...         ...
376    ufind rreveal    0.003350
380       ur awarded    0.003348
86   congrats mobile    0.003213
390   ur specialcall    0.003139
2           100 free    0.001803

[82 rows x 2 columns]


**Tworzymy nowe modele Random Forest z wyborem hiperparametrów korzystając z GridSearch**

In [13]:
params_tfidf = {'vectorizer_tfidf__ngram_range': [(1, 1), (1, 2), (2, 2), (2, 3), (3, 3)],
               'vectorizer_tfidf__max_df':[0.4, 0.5, 0.6],
               'vectorizer_tfidf__min_df':[0.0005, 0.001, 0.002],
               'classifier_rf_tfidf__max_depth':[1, 2, 3]
}

grid_search_rf_tfidf = GridSearchCV(model_pipeline_tfidf,
                                    params_tfidf,
                                    cv=5,
                                    n_jobs=-1,
                                    verbose=10,
                                    scoring='accuracy'
)

grid_search_rf_tfidf.fit(X_train, y_train)

best_model_pipeline_tfidf = grid_search_rf_tfidf.best_estimator_

print("Najlepsze parametry dla grid_search_rf_tfidf:", grid_search_rf_tfidf.best_params_)
print("Najlepszy wynik (accuracy) dla grid_search_rf_tfidf na zbiorze walidacyjnym (kross validacja):", grid_search_rf_tfidf.best_score_)

Fitting 5 folds for each of 135 candidates, totalling 675 fits
Najlepsze parametry dla grid_search_rf_tfidf: {'classifier_rf_tfidf__max_depth': 3, 'vectorizer_tfidf__max_df': 0.4, 'vectorizer_tfidf__min_df': 0.002, 'vectorizer_tfidf__ngram_range': (3, 3)}
Najlepszy wynik (accuracy) dla grid_search_rf_tfidf na zbiorze walidacyjnym (kross validacja): 0.8703167700925547


In [14]:
params_count = {'vectorizer_count__ngram_range': [(1, 1), (1, 2), (2, 2), (2, 3), (3, 3)],
               'vectorizer_count__max_df':[0.4, 0.5, 0.6],
               'vectorizer_count__min_df':[0.0005, 0.001, 0.002],
               'classifier_rf_count__max_depth':[1, 2, 3]
}

grid_search_rf_count = GridSearchCV(model_pipeline_count,
                                    params_count,
                                    cv=5,
                                    n_jobs=-1,
                                    verbose=10,
                                    scoring='accuracy'
)

grid_search_rf_count.fit(X_train, y_train)

best_model_pipeline_count = grid_search_rf_count.best_estimator_

print("Najlepsze parametry dla grid_search_rf_count:", grid_search_rf_count.best_params_)
print("Najlepszy wynik (accuracy) dla grid_search_rf_count na zbiorze walidacyjnym (kross validacja):", grid_search_rf_count.best_score_)

Fitting 5 folds for each of 135 candidates, totalling 675 fits
Najlepsze parametry dla grid_search_rf_count: {'classifier_rf_count__max_depth': 3, 'vectorizer_count__max_df': 0.4, 'vectorizer_count__min_df': 0.002, 'vectorizer_count__ngram_range': (3, 3)}
Najlepszy wynik (accuracy) dla grid_search_rf_count na zbiorze walidacyjnym (kross validacja): 0.8703167700925547


In [15]:
print("***PODSUMOWANIE***\n")
print("📊 TF-IDF — model przed GridSearchCV:")
print("Dokładność na zbiorze testowym:", round(model_pipeline_tfidf.score(X_test, y_test), 4))

print("\n✅ TF-IDF — najlepszy model po GridSearchCV:")
print("Dokładność na zbiorze testowym:", round(best_model_pipeline_tfidf.score(X_test, y_test), 4))


print("\n📊 CountVectorizer — model przed GridSearchCV:")
print("Dokładność na zbiorze testowym:", round(model_pipeline_count.score(X_test, y_test), 4))

print("\n✅ CountVectorizer — najlepszy model po GridSearchCV:")
print("Dokładność na zbiorze testowym:", round(best_model_pipeline_count.score(X_test, y_test), 4))

***PODSUMOWANIE***

📊 TF-IDF — model przed GridSearchCV:
Dokładność na zbiorze testowym: 0.8664

✅ TF-IDF — najlepszy model po GridSearchCV:
Dokładność na zbiorze testowym: 0.8753

📊 CountVectorizer — model przed GridSearchCV:
Dokładność na zbiorze testowym: 0.8664

✅ CountVectorizer — najlepszy model po GridSearchCV:
Dokładność na zbiorze testowym: 0.8753
